 ### Importing Required Libraries
 

In [ ]:
import numpy as np # linear algebra
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pre_app = pd.read_csv('../input/loan-defaulter/previous_application.csv')
pre_app.head()


In [ ]:
app_data = pd.read_csv('../input/loan-defaulter/application_data.csv')
app_data.head()

In [ ]:
app_data.info()

### Shape of dateframes

In [ ]:
pre_app.shape

In [ ]:
app_data.shape

In [ ]:
# Details of each column contained in application_data.csv
app_data.describe(include='all')

## Checking columns having missing values greater than 50 %

In [ ]:
# Checking for null values in dataframe, df

print("------------Percentage of Nulls in application_data -------------------------")
round(100*(app_data.isnull().sum()/len(app_data.index)),2)

#### Dropping the columns having missing values greater than 50 %


In [ ]:
(app_data.isnull().sum() * 100 / len(app_data)).sort_values(ascending = False)

In [ ]:
app_data.columns[(100*app_data.isna().sum()/len(app_data) > 50)]

In [ ]:
app_data= app_data.drop(columns=app_data.columns[(100*app_data.isna().sum()/len(app_data) > 50)])

In [ ]:
app_data.shape ## Rechecking the shape of data

In [ ]:
#app_data[app_data.AMT_ANNUITY.isnull()]

## Imputing missing values

1. AMT_GOODS_PRICE has 0.09% of null values. It is observed that AMT_GOODS_PRICE is null when NAME_CONTRACT_TYPE = Revolving loans. While in case when we do have a value it is found to be equal to AMT_CREDIT. Hence, we will impute data for missing values of AMT_GOODS_PRICE with corresponding AMT_CREDIT value.



In [ ]:
app_data.loc[pd.isnull(app_data['AMT_GOODS_PRICE']),'AMT_GOODS_PRICE'] = app_data['AMT_CREDIT']

In [ ]:
#checking if AMT_GOODS_PRICE null values are imputed

sum(pd.isnull(app_data['AMT_GOODS_PRICE']))

2. NAME_TYPE_SUITE has 0.42% of null values. Since a person can be either accompanied with someone or Unaccompanied. we will fill the NAN values with the most recurring or mode value of this data. 

In [ ]:
 app_data.NAME_TYPE_SUITE.isna().sum()

In [ ]:
 app_data.NAME_TYPE_SUITE.value_counts()

In [ ]:
NAME_TYPE_SUITE_Mode = app_data.NAME_TYPE_SUITE.mode()[0]
NAME_TYPE_SUITE_Mode

In [ ]:
app_data.NAME_TYPE_SUITE.fillna(NAME_TYPE_SUITE_Mode,inplace=True)

In [ ]:
#checking if NAME_TYPE_SUITE null values are imputed
app_data.NAME_TYPE_SUITE.isna().sum()

In [ ]:
 app_data.OCCUPATION_TYPE.isna().sum()

In [ ]:
 app_data.OCCUPATION_TYPE.value_counts()

In [ ]:
app_data.loc[pd.isnull(app_data['OCCUPATION_TYPE']),'OCCUPATION_TYPE'] = "Others"

4. Filling null values as median for AMT_ANNUITY column as there are many outliers in it.

In [ ]:
plt.figure(figsize=(6,6))
sns.boxplot(app_data.AMT_ANNUITY)
plt.show()

In [ ]:
app_data['AMT_ANNUITY'] = app_data['AMT_ANNUITY'].fillna(app_data['AMT_ANNUITY'].median())

In [ ]:
app_data['AMT_ANNUITY'].median()

5. Handling missing values in code gender

In [ ]:
app_data['CODE_GENDER'].value_counts()

Since, there are 4 null values we are going to drop those as they don't skew the data.

In [ ]:

# Dropping the fields in 'CODE_GENDER' with "XNA" values for the dataset

app_data=app_data[app_data['CODE_GENDER']!='XNA']
app_data['CODE_GENDER'].value_counts()

In [ ]:
app_data=app_data.drop(app_data.loc[app_data['ORGANIZATION_TYPE']=='XNA'].index)

In [ ]:
app_data[app_data['ORGANIZATION_TYPE']=='XNA'].shape

In [ ]:
# Dropping the fields in 'NAME_FAMILY_STATUS' with "Unknown" values for the dataset
app_data=app_data.drop(app_data.loc[app_data['NAME_FAMILY_STATUS']=='Unknown'].index)

In [ ]:
app_data['NAME_FAMILY_STATUS'].value_counts()

In [ ]:
# Removing the column values of 'XNA' and 'XAP' in prevoius application data.
pre_app = pre_app.drop(pre_app[pre_app['NAME_CASH_LOAN_PURPOSE']=='XNA'].index)
pre_app = pre_app.drop(pre_app[pre_app['NAME_CASH_LOAN_PURPOSE']=='XAP'].index)

In [ ]:
# Since fields like days employed, etc cannot be negative, we will make them positive.

app_data['DAYS_BIRTH']=app_data.DAYS_BIRTH.abs()
app_data['DAYS_EMPLOYED']=app_data.DAYS_EMPLOYED.abs()
app_data['DAYS_ID_PUBLISH']=app_data.DAYS_ID_PUBLISH.abs()
app_data['DAYS_LAST_PHONE_CHANGE']=app_data.DAYS_LAST_PHONE_CHANGE.abs()
app_data['DAYS_REGISTRATION']=app_data.DAYS_REGISTRATION.abs()


In [ ]:
#Calculating age in years from days of birth.
app_data['AGE'] = app_data['DAYS_BIRTH'].apply(lambda x:x/365)

In [ ]:
#Detecting Outliers in days employed
plt.figure(figsize=(6,6))
sns.boxplot(app_data.DAYS_EMPLOYED)
plt.show()

It is clearly visible that there is some mistake in the data as nobody can work more than 350000 days in a life time and
they are outliers. We will be dropping these values.

In [ ]:
app_data=app_data[app_data.DAYS_EMPLOYED<50000]

In [ ]:
plt.figure(figsize=(6,6))
sns.boxplot(app_data.DAYS_EMPLOYED)
plt.show()

In [ ]:
#Calculating years employed from days of eployment.
app_data['YEARS_EMPLOYED'] = app_data['DAYS_EMPLOYED'].apply(lambda x:x/365)

In [ ]:
#Using countplot to understand the OCCUPATION_TYPE
plt.figure(figsize=(9,6))
sns.countplot(x = 'OCCUPATION_TYPE', data=app_data,palette='dark',)
plt.title("Count of Clients based on their Occupation Type",size=15)
plt.xticks(rotation = 90);

Labourers make up the most number of clients, followed by occupation in "Others"

In [ ]:
# Checking Distribution of Target Variable
ax=app_data['TARGET'].value_counts(normalize=True)
labels='Non-Defaulters','Defaulters'
plt.pie(ax,labels=labels,autopct='%1.1f%%')
plt.title("Distribution of Clients based on Target variable ")
plt.show()

Only close to 9% of applicants are likely to default.

In [ ]:
ax=app_data['NAME_CONTRACT_TYPE'].value_counts(normalize=True)
plt.pie(ax,labels=ax.index,autopct='%1.1f%%')
plt.title("Distribution of Clients based on Contract Type ")
plt.show()

Approx. 90% loans are cash loans

In [ ]:
fig = plt.figure(figsize=(4,6))
g=sns.countplot(app_data.TARGET)
plt.xlabel("Client History")
plt.ylabel("Count")
g.set_xticklabels(['Non-Defaulters','Defaulters'])
plt.title("Distribution of TARGET Variable")
for p in g.patches:
    g.annotate(str(p.get_height()) , (p.get_x() + p.get_width() / 2, p.get_height()* 1.009),
                ha='center', fontsize=11)

plt.show()

In [ ]:
plt.figure(figsize=(10,6))
ax=sns.countplot(data=app_data,x='TARGET',hue='CODE_GENDER')
plt.xlabel("Defaulters vs Non-Defaulters")
plt.title("Gender based analysis of target Variable")
for p in ax.patches:        
        height = p.get_height()
        ax.text(p.get_x() + p.get_width()/2., height, height, fontsize=12, ha='center', va='bottom')

Females tend to take more loans and are also less likely to default.

In [ ]:
plt.figure(figsize=(10,6))
ax=sns.countplot(data=app_data,x='NAME_CONTRACT_TYPE',hue='TARGET')
plt.xlabel("Type of Loan for current application")
plt.title("Distribution of Loan Type based on Default Status")
# annoting labels
for p in ax.patches:        
        height = p.get_height()
        ax.text(p.get_x() + p.get_width()/2., height, height, fontsize=12, ha='center', va='bottom')

1. Cash loans are most likey to default.

In [ ]:
#Calculating the imbalance percentage of the TARGET column
Imbalance_ratio=round(app_data['TARGET'].value_counts()[1]/app_data['TARGET'].value_counts()[0]*100,2)
print("Imbalance percentage of the Target column = {}%".format(Imbalance_ratio))


#### It is clearly visible that most of the loans requested are cash loans. However Revloving loans are less likely to default.

In [ ]:
bins = [0,25000,50000,75000,100000,125000,150000,175000,200000,225000,250000,275000,300000,325000,350000,375000,400000,425000,450000,475000,500000,10000000000]
slot = ['0-25000', '25000-50000','50000-75000','75000,100000','100000-125000', '125000-150000', '150000-175000','175000-200000',
       '200000-225000','225000-250000','250000-275000','275000-300000','300000-325000','325000-350000','350000-375000',
       '375000-400000','400000-425000','425000-450000','450000-475000','475000-500000','500000 and above']

app_data['AMT_INCOME_TOTAL_CAT'] = pd.cut(app_data['AMT_INCOME_TOTAL'],bins,labels=slot)

In [ ]:
# Dividing the dataset into two datasets of target=1(customer with defaults) and target=0(all other)
defaulters=app_data[app_data.TARGET==1]
nondefaulters=app_data[app_data.TARGET==0]

In [ ]:
plt.figure(figsize=(10,6))
sns.set(style="whitegrid")
sns.countplot(data =nondefaulters, x='AMT_INCOME_TOTAL_CAT',hue='CODE_GENDER')
plt.title('Distribution of total income for Non-Defaulters',size=15)
plt.xticks(rotation=45)
plt.show()

Inference:

1. Females generally have higher incomes than males up untill 275000.
    

In [ ]:
plt.figure(figsize=(10,6))
sns.set(style="whitegrid")
ax=sns.countplot(data =nondefaulters, x='NAME_CONTRACT_TYPE',hue='CODE_GENDER')
plt.title('Distribution of Contract Type for Non-Defaulters')
plt.xticks(rotation=45)
for p in ax.patches:        
        height = p.get_height()
        ax.text(p.get_x() + p.get_width()/2., height, height, fontsize=12, ha='center', va='bottom')
plt.show()

Inference:

1. Females form the majority among non-defaulters as well as in both types of loan contracts
2. Cash loans are prefferred choice of loan requested both among males and females.

In [ ]:
plt.figure(figsize=(15,6))
sns.set(style="whitegrid")
ax=sns.countplot(data =nondefaulters, x='OCCUPATION_TYPE',hue='CODE_GENDER')
plt.title('Distribution of Occupation Type for Non-Defaulters')
plt.xticks(rotation=45)
plt.show()

1. Males have highest concentration in driver occupation.
2. Females make a majority in the medical staff profession.

In [ ]:
plt.figure(figsize=(15,6))
sns.set(style="whitegrid")
ax=sns.countplot(data =app_data, x='OCCUPATION_TYPE',hue='TARGET')
plt.title('Distribution of Occupation Type for Defaulters vs Non-Defaulters')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(15,20))
sns.countplot(data=nondefaulters,y='ORGANIZATION_TYPE',order=nondefaulters['ORGANIZATION_TYPE'].value_counts().index,palette='copper_r')
plt.show()

People in Business Entity Type 3 and those who are Self-employed are amongst the highest credit seekers.

In [ ]:
plt.figure(figsize=(10,6))
sns.set(style="whitegrid")
sns.countplot(data =defaulters, x='AMT_INCOME_TOTAL_CAT',hue='CODE_GENDER')
plt.title('Distribution of total income for Defaulters')
plt.xticks(rotation=45)
plt.show()

Inference:

1. Females with total income less than 150000 are more likely to default than males.

In [ ]:
# Now merging the Application dataset with previous appliaction dataset

new_df=pd.merge(left=app_data,right=pre_app,how='inner',on='SK_ID_CURR',suffixes='_x')


In [ ]:
# Removing unwanted columns in new dataframe
new_df.drop(['SK_ID_CURR','REG_REGION_NOT_LIVE_REGION', 
              'REG_REGION_NOT_WORK_REGION','LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
              'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY',
             'FLAG_LAST_APPL_PER_CONTRACT','NFLAG_LAST_APPL_IN_DAY'],axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(15,30))
sns.countplot(data =new_df, y='NAME_CASH_LOAN_PURPOSE',order = new_df['NAME_CASH_LOAN_PURPOSE'].value_counts().index, hue='NAME_CONTRACT_STATUS')
plt.show()

In [ ]:
plt.figure(figsize=(15,30))
sns.countplot(data =new_df, y='NAME_CASH_LOAN_PURPOSE',order = new_df['NAME_CASH_LOAN_PURPOSE'].value_counts().index, hue='NAME_CONTRACT_STATUS')
plt.show()

In [ ]:
new_df.head()

1. Highest number of loan rejections are for “Repair” and “Other” purposes.
2. Education Loans have a 50% success rate.
3. Almost all categories have a higher rejection rate than approval except for purchase of electronic equipment.

In [ ]:
#Correlation Matrix for Numeric Variables
loan_corr =new_df.corr().round(3)
loan_corr

In [ ]:
#plotting correlation
f, ax = plt.subplots(figsize=(75, 45))
sns.heatmap(loan_corr, 
            xticklabels=loan_corr.columns.values,
            yticklabels=loan_corr.columns.values,annot= True)
plt.show()

1. Credit amount on the previous application(AMT_CREDITx) has a high correlation with AMT_APPLICATION at 0.99 meaning the loan amount is 99% of the time equal to what is requested by the applicant.
2. Goods price has a strong correlation with the credit amount on the previous loan. It means that clients like to fully finance the goods using loans.

In [ ]:
# AGE
plt.figure(figsize=(20,10))
sns.set(font_scale=1.4)

sns.distplot(nondefaulters['AGE'], hist = False, label='Non-Defaulters')
sns.distplot(defaulters['AGE'], hist = False,label='Defaulters')
plt.legend()
plt.show()

Clients in the age group 25- 40 are more likely are to default on a loan.

In [ ]:
# YEARS_EMPLOYED
plt.figure(figsize=(20,10))
sns.set(font_scale=1.4)

sns.distplot(nondefaulters['YEARS_EMPLOYED'], hist = False,label='Non-Defaulters')
sns.distplot(defaulters['YEARS_EMPLOYED'], hist = False,label='Defaulters')
plt.legend()
plt.show()

People with less than 5 years of employment are more likley to default on a loan.

In [ ]:
# Relationship between NAME_FAMILY_STATU and TARGET variable
sns.countplot(x='NAME_FAMILY_STATUS',hue='TARGET',data=app_data)
plt.legend(['Non-Defaulters', 'Defaulters'])
plt.xticks(rotation=90)
plt.show()

Applicant with Married family status are less likely to default.

In [ ]:
# Relationship between NAME_HOUSING_TYPE and TARGET variable
sns.countplot(x='NAME_HOUSING_TYPE',hue='TARGET',data=app_data)
plt.legend(['Non-Defaulters', 'Defaulters'])
plt.xticks(rotation=90)
plt.show()

Applicants who own a house/apartment are less likely to default.

In [ ]:
# Relationship between NAME_EDUCATION_TYPE and TARGET variable
sns.countplot(x='NAME_EDUCATION_TYPE',hue='TARGET',data=app_data)
plt.legend(['Non-Defaulters', 'Defaulters'])
plt.xticks(rotation=90)
plt.show()

Applicants with Secondary special education are less likely to default.